In [43]:
import json
import pandas as pd
  
df = pd.read_json("../../coordinates.json")

df[['longitude','latitude']] = df.coordinates.apply(lambda x: pd.Series(str(x).split(",")))
df = df.drop(columns=['coordinates'])


In [20]:
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)

In [21]:
import requests
weather = []
for index, row in df.iterrows():
    appid = '7e208a67c297443752c742b0cdd73fdf'
    lat= row['latitude']
    lon= row['longitude']
    city = row['city']
    # URL = 'https://api.openweathermap.org/data/3.0/onecall'
    # PARAMS = {'lat':lat, 'lon':lon, 'appid': appid, 'units': 'metrics', 'cnt':7, 'exclude':'urrent,minutely,hourly,alerts'}
    # r = requests.get(url = URL, params=PARAMS).json()
    URL = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&exclude=current,minutely,hourly,alerts&appid={appid}'
    r = requests.get(URL).json()
    for i in range(7):
        weather.append({'city':city,'lat':lat, 'lon':lon, 'day': r['daily'][i]['dt'], 'temp':r['daily'][i]['temp']['day']})

    

In [22]:

with open('weather.json', 'w') as f:
    f.write(json.dumps(weather, indent=2))


In [23]:
weather_data = pd.read_json("weather.json")
weather_data.head(7)

,city,lat,lon,day,temp
0,Mont Saint-Michel,48.635954,-1.51146,1667559600,13.29
1,Mont Saint-Michel,48.635954,-1.51146,1667646000,11.25
2,Mont Saint-Michel,48.635954,-1.51146,1667732400,14.77
3,Mont Saint-Michel,48.635954,-1.51146,1667818800,16.05
4,Mont Saint-Michel,48.635954,-1.51146,1667905200,14.35
5,Mont Saint-Michel,48.635954,-1.51146,1667991600,14.07
6,Mont Saint-Michel,48.635954,-1.51146,1668078000,15.03


In [24]:
weather_data['temp_mean'] = weather_data['temp'].groupby(weather_data['lat']).transform('mean')



In [25]:
weather_data = weather_data.drop(columns=['temp'])
weather_data = weather_data.drop(columns=['day'])
weather_data.round({'temp_mean': 1})

,city,lat,lon,temp_mean
0,Mont Saint-Michel,48.635954,-1.511460,14.1
1,Mont Saint-Michel,48.635954,-1.511460,14.1
2,Mont Saint-Michel,48.635954,-1.511460,14.1
3,Mont Saint-Michel,48.635954,-1.511460,14.1
4,Mont Saint-Michel,48.635954,-1.511460,14.1
...,...,...,...,...
240,La Rochelle,46.159113,-1.152043,15.7
241,La Rochelle,46.159113,-1.152043,15.7
242,La Rochelle,46.159113,-1.152043,15.7
243,La Rochelle,46.159113,-1.152043,15.7


In [26]:
weather_data = weather_data.drop_duplicates(subset=['lat', 'lon'])
weather_data = weather_data.reset_index(drop=True)

In [27]:
weather_data

,city,lat,lon,temp_mean
0,Mont Saint-Michel,48.635954,-1.511460,14.115714
1,Saint-Malo,48.649518,-2.026041,14.604286
2,Bayeux,49.276462,-0.702474,13.441429
3,Le Havre,49.493898,0.107973,13.725714
4,Rouen,49.440459,1.093966,13.064286
5,Paris,48.858890,2.320041,13.137143
6,Amiens,49.894171,2.295695,12.672857
7,Lille,50.636565,3.063528,12.250000
8,Strasbourg,48.584614,7.750713,12.615714
9,Ch\u00e2teau du Haut-K\u0153nigsbourg,48.249490,7.344296,9.835714


In [28]:
weather_data = weather_data.sort_values(by='temp_mean', ascending=False)

In [29]:
top_five_weather = weather_data.head(5)

In [30]:
top_five_weather.round({'temp_mean': 1})

,city,lat,lon,temp_mean
19,Cassis,43.214036,5.539632,18.1
20,Marseille,43.296174,5.369953,17.8
27,Collioure,42.525050,3.083155,17.7
18,Bormes-les-Mimosas,43.150697,6.341928,17.6
25,Aigues-Mortes,43.565823,4.191284,17.2


In [31]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter_mapbox(top_five_weather, lat="lat", lon="lon", color="temp_mean", size="temp_mean", hover_name="city", zoom=6, mapbox_style="carto-positron")
fig.show()

In [32]:
top_city_names = top_five_weather['city']
top_city_names.to_string('top_city_names.txt',index = False)

In [33]:
top_city_names

19                Cassis
20             Marseille
27             Collioure
18    Bormes-les-Mimosas
25         Aigues-Mortes
Name: city, dtype: object

In [34]:
top_city_names.to_csv('top_5_city_names.csv', index=False) 